# Задачи от Х5 по SQL

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import string
con = sqlite3.connect('retail')

## Задача 1

Скрипт, который создает таблицу с визитами клиентов на сайт:   
клиент, дата, кол-во сессий и общая длительность сессий за день.

In [2]:
N=100000

In [3]:
day= np.random.randint(1,29,N)
month=np.random.randint(1,13,N)
year=[2022 for i in range(N)]

In [4]:
str(day[0])

'26'

In [5]:

client_id = np.random.randint(1,2000, N)
date = [str(year[i])+'-'+str(month[i])+'-'+str(day[i]) for i in range(N)]
session_count = np.random.randint(1,60, N)
session_time=[session_count[i] * np.random.randint(1,60) for i in range(N)]

In [6]:
data=pd.DataFrame({'client_id': client_id , 
                  'date': date,
                   'session_count': session_count,
                   'session_time' : session_time
                  })

In [7]:
data.head()

,client_id,date,session_count,session_time
0,926,2022-7-26,23,644
1,1107,2022-12-14,30,450
2,885,2022-2-11,48,1296
3,276,2022-11-19,6,348
4,530,2022-8-14,12,120


In [8]:
data.date = data.date.astype('datetime64')

In [9]:
data.dtypes

client_id                 int32
date             datetime64[ns]
session_count             int32
session_time              int32
dtype: object

In [10]:


data.to_sql('data',con, index=False, if_exists='replace')

## Задача 2 

За каждый день выведите DAU и % изменения относительно предыдущего дня   
(день, DAU, % изменения).   
В результате:   
Колонка с датой (от меньшего к большему), DAU, Prev_Change - % изменения DAU относительно предыдущего дня

In [11]:
sql1='''
SELECT
    *
FROM
    data
'''

In [12]:
sel1= pd.read_sql(sql1,con)

In [13]:
sel1.head()

,client_id,date,session_count,session_time
0,926,2022-07-26 00:00:00,23,644
1,1107,2022-12-14 00:00:00,30,450
2,885,2022-02-11 00:00:00,48,1296
3,276,2022-11-19 00:00:00,6,348
4,530,2022-08-14 00:00:00,12,120


In [14]:
sql2= '''
select 
    date,
    DAU,
    100 * ( cast(DAU as float ) /cast( PreviousDayDAU as float )  - 1) as prev_change
from (
        select 
            date,
            count(distinct client_id) as DAU,
            LAG ( count(distinct client_id), 1, 0 ) OVER ( 
                ORDER BY date 
                )      as PreviousDayDAU
        From
            data


        Group by
            date

        ORDER BY date
) as sq    
    

    


'''

In [15]:
sel2= pd.read_sql(sql2,con)
sel2.head(10)

,date,DAU,prev_change
0,2022-01-01 00:00:00,268,NaN
1,2022-01-02 00:00:00,256,-4.477612
2,2022-01-03 00:00:00,268,4.687500
3,2022-01-04 00:00:00,252,-5.970149
4,2022-01-05 00:00:00,245,-2.777778
5,2022-01-06 00:00:00,277,13.061224
6,2022-01-07 00:00:00,261,-5.776173
7,2022-01-08 00:00:00,259,-0.766284
8,2022-01-09 00:00:00,290,11.969112
9,2022-01-10 00:00:00,283,-2.413793


## Задание 3

Выведите визиты ТОП10 клиентов по их времяпрепровождению на сайте   
за весь период

In [16]:
sql3='''
select 
    client_id,
    sum(session_time) as client_total_time

from 
    data


group by 
    client_id

order by
    client_total_time desc

limit 10

'''

In [17]:
sel3= pd.read_sql(sql3,con)
sel3.head(10)

,client_id,client_total_time
0,1533,81677
1,499,76788
2,1474,74790
3,1084,72094
4,1357,70702
5,340,69814
6,609,69526
7,1968,69500
8,1196,69190
9,1185,69102
